In [12]:
import pandas as pd
import numpy as np
# from tqdm import tqdm # Removed tqdm import
def merge_intervals(intervals):
    if not intervals:
        return []

    # Сортировка по начальному времени
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for current in intervals[1:]:
        prev_start, prev_end = merged[-1]
        curr_start, curr_end = current

        # Проверка на перекрытие или соприкосновение
        if curr_start <= prev_end:
            # Объединяем интервалы
            merged[-1] = (prev_start, max(prev_end, curr_end))
        else:
            merged.append(current)

    return merged

def merge_intervals_with_gap(intervals, gap_threshold=0.5):
    """
    Merges overlapping or closely spaced intervals based on a gap threshold.

    Args:
        intervals: A list of intervals as tuples (start, end).
        gap_threshold: The maximum allowed gap between intervals to be merged.

    Returns:
        A list of merged intervals.
    """
    if not intervals:
        return []

    # Sort intervals by start time
    intervals.sort(key=lambda x: x[0])

    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        prev_start, prev_end = merged[-1]

        # Check for overlap or if the gap is within the threshold
        if current_start <= prev_end or (current_start - prev_end) <= gap_threshold:
            # Merge intervals
            merged[-1] = (prev_start, max(prev_end, current_end))
        else:
            # Append as a new interval
            merged.append((current_start, current_end))

    return merged

def calculate_iou(interval_a, interval_b):
    start_a, end_a = interval_a
    start_b, end_b = interval_b

    intersection = max(0, min(end_a, end_b) - max(start_a, start_b))
    union = (end_a - start_a) + (end_b - start_b) - intersection
    return intersection / union if union > 0 else 0.0

def calculate_ap(true_segments, pred_segments, iou_thresholds):
    aps = []

    for thresh in iou_thresholds:
        # Стандартный расчет TP, FP для AP
        # AP метрики обычно основаны на сопоставлении один-к-одному или один-ко-многим
        # с учетом ранжирования предсказаний (по уверенности), которого у нас нет.
        # Здесь используется стандартный расчет TP/FP для AP для каждого порога IoU.
        tp_ap = np.zeros(len(pred_segments))
        fp_ap = np.zeros(len(pred_segments))
        matched_true_indices_ap = set()

        for i, pred in enumerate(pred_segments):
            best_iou = 0.0
            best_true_idx = -1
            for j, true in enumerate(true_segments):
                # При стандартном подходе истинный интервал может быть сопоставлен только один раз
                if j in matched_true_indices_ap:
                    continue
                iou = calculate_iou(pred, true)
                if iou > best_iou:
                    best_iou = iou
                    best_true_idx = j

            if best_iou >= thresh:
                matched_true_indices_ap.add(best_true_idx)
                tp_ap[i] = 1
            else:
                fp_ap[i] = 1

        # Рассчитываем precision-recall кривую для стандартного AP
        tp_cumsum_ap = np.cumsum(tp_ap)
        fp_cumsum_ap = np.cumsum(fp_ap)

        recalls_ap = tp_cumsum_ap / len(true_segments) if len(true_segments) > 0 else np.zeros_like(tp_cumsum_ap)
        precisions_ap = tp_cumsum_ap / (tp_cumsum_ap + fp_cumsum_ap + 1e-12)

        # Интерполяция precision для 101 точки для стандартного AP
        interp_precision_ap = np.zeros(101)
        for r in range(101):
            precision_vals_ap = precisions_ap[recalls_ap >= r/100]
            interp_precision_ap[r] = max(precision_vals_ap) if len(precision_vals_ap) > 0 else 0

        # Вычисляем AP как среднее значение precision для стандартного AP
        ap_score = np.mean(interp_precision_ap)
        aps.append(ap_score)

    return aps


def evaluate_metrics_modified(true_segments, pred_segments, iou_threshold=0.2):
    """
    Evaluates metrics allowing one predicted interval to match multiple true intervals.

    Args:
        true_segments: List of true intervals (start, end).
        pred_segments: List of predicted intervals (start, end).
        iou_threshold: IoU threshold for considering a match.

    Returns:
        A dictionary of calculated metrics (Precision, Recall, F1-score, AP50, AP75, AP50-95).
    """
    # Новый расчет TP, FP, FN для Precision/Recall/F1
    # Истинный позитив: истинный интервал, который пересекается с хотя бы одним предсказанным выше порога.
    # Ложный позитив: предсказанный интервал, который не пересекается ни с одним истинным выше порога.
    # Ложный негатив: истинный интервал, который не пересекается ни с одним предсказанным выше порога.

    detected_true_count = 0
    matched_pred_indices_for_fn_fp = set() # Для отслеживания предсказаний, которые совпали с истинными

    for true_interval in true_segments:
        is_true_detected = False
        for pred_idx, pred_interval in enumerate(pred_segments):
            if calculate_iou(true_interval, pred_interval) >= iou_threshold:
                is_true_detected = True
                matched_pred_indices_for_fn_fp.add(pred_idx)
                # Не прерываем, так как один истинный может пересекаться с несколькими предсказанными
        if is_true_detected:
            detected_true_count += 1

    true_positives = detected_true_count
    false_negatives = len(true_segments) - true_positives

    # Предсказанные интервалы, которые не совпали ни с одним истинным, являются ложными позитивами
    false_positives = 0
    for pred_idx in range(len(pred_segments)):
        if pred_idx not in matched_pred_indices_for_fn_fp:
            false_positives += 1

    #print(f"TP: {true_positives}, FN: {false_negatives}, FP: {false_positives}")

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    # AP metrics remain based on the standard one-to-one matching for PR curve calculation
    ap50 = calculate_ap(true_segments, pred_segments, [0.5])[0]
    ap75 = calculate_ap(true_segments, pred_segments, [0.75])[0]
    ap50_95 = np.mean(calculate_ap(true_segments, pred_segments, np.arange(0.5, 1.0, 0.05)))

    return {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1_score,
        'AP50': ap50,
        'AP75': ap75,
        'AP50-95': ap50_95
    }


def bootstrap_confidence_intervals_modified(true_segments, pred_segments, iou_threshold=0.2,
                                  n_bootstrap=1000, confidence_level=0.95):
    """
    Calculates bootstrap confidence intervals using the modified evaluation metrics.
    """
    n_true = len(true_segments)
    n_pred = len(pred_segments)

    metrics_samples = {
        'Precision': [],
        'Recall': [],
        'F1-score': [],
        'AP50': [],
        'AP75': [],
        'AP50-95': []
    }

    for _ in range(n_bootstrap): # Removed tqdm
        # Bootstrap с заменой из исходных выборок
        true_bootstrap_indices = np.random.choice(n_true, n_true, replace=True)
        pred_bootstrap_indices = np.random.choice(n_pred, n_pred, replace=True)

        true_bootstrap = [true_segments[i] for i in true_bootstrap_indices]
        pred_bootstrap = [pred_segments[i] for i in pred_bootstrap_indices]

        # Используем модифицированную функцию оценки метрик
        metrics = evaluate_metrics_modified(true_bootstrap, pred_bootstrap, iou_threshold)
        for k in metrics_samples:
            metrics_samples[k].append(metrics[k])

    # Вычисление квантилей для всех метрик
    alpha = (1 - confidence_level) / 2
    ci = {}

    for metric, samples in metrics_samples.items():
        ci[metric + "_CI"] = np.percentile(samples, [alpha*100, (1-alpha)*100])

    return ci

# Предполагается, что detections и true_label.txt уже загружены и обработаны
# как в исходном коде для получения true_intervals и predicted_intervals (после merge_intervals, если используете его)

# Пример использования с модифицированной функцией:

# Загрузка данных (как в исходном коде)
try:
    detections = pd.read_csv('../dataset/detections.csv')
    detections_call = detections[detections['label'] == 'call']
    # Использование оригинальной merge_intervals или новой merge_intervals_with_gap
    # Если вы хотите использовать объединенные интервалы для предсказаний:
    # predicted_intervals_for_eval = merge_intervals_with_gap(list(zip(detections_call['start_s'], detections_call['end_s']))) # Используем merge_intervals_with_gap
    # Если вы хотите использовать необъединенные интервалы для предсказаний:
    predicted_intervals_for_eval = (list(zip(detections_call['start_s'], detections_call['end_s'])))


    true_intervals = []
    with open('../dataset/true_label.txt', 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) < 2:
                continue
            start_str = parts[0].replace(',', '.')
            end_str = parts[1].replace(',', '.')
            start = float(start_str)
            end = float(end_str)
            true_intervals.append((start, end))

    print(true_intervals, "\n" ,detections_call)
    # IoU порог для оценки (можно настроить)
    iou_global = 0.5

    # Оценка метрик с модифицированной функцией
    metrics_modified = evaluate_metrics_modified(true_intervals, predicted_intervals_for_eval, iou_threshold=iou_global)

    print("\nMetrics (Modified Evaluation Logic):")
    for metric, value in metrics_modified.items():
        print(f"{metric}: {value:.4f}")

    # Расчет бутстрап доверительных интервалов с модифицированной функцией оценки
    # Установите n_bootstrap на меньшее число, например 100, для более быстрого выполнения, если необходимо.
    bootstrap_ci_modified = bootstrap_confidence_intervals_modified(true_intervals, predicted_intervals_for_eval, iou_threshold=iou_global, n_bootstrap=1000)

    print("\nBootstrap Confidence Intervals (Modified Evaluation Logic):")
    for metric, ci_values in bootstrap_ci_modified.items():
        print(f"{metric}: [{ci_values[0]:.4f} {ci_values[1]:.4f}]")

except FileNotFoundError:
    print("Error: dataset files not found. Please make sure 'detections.csv' and 'true_label.txt' are in the '../dataset/' directory.")
except Exception as e:
    print(f"An error occurred: {e}")

[(16.64595, 17.64888), (24.368349, 25.349689), (42.17569, 42.778351), (43.80769, 44.47702), (41.309021, 42.06636), (51.514622, 52.214931), (61.05003, 61.72472), (75.898003, 77.025993), (84.827797, 85.286003), (109.514198, 110.776497), (116.905998, 117.618103), (128.7603, 129.434601), (131.129303, 131.864105), (134.044998, 134.736801), (158.687897, 159.206604), (167.8526, 168.423004), (198.684097, 199.358398), (203.811203, 204.407898), (252.985703, 253.841599), (49.790119, 50.428871), (157.111206, 157.822998), (158.680695, 159.958206), (160.797699, 161.746704), (172.183502, 172.779694), (186.304596, 187.077194), (187.624695, 188.518906), (189.139404, 189.723404), (222.542496, 223.023102), (254.650299, 255.349899), (279.752411, 280.257294)] 
      start_s  end_s label     score
33      16.5   17.5  call  0.998833
34      17.0   18.0  call  0.982521
48      24.0   25.0  call  0.996545
49      24.5   25.5  call  0.998436
82      41.0   42.0  call  0.990721
..       ...    ...   ...       .